#### Construction de la base d'apprentissage

In [ ]:
#Pour construire la base d'apprentissage seulement
#from Bio import SeqIO
#import RNA

In [1]:
import numpy as np
import pandas as pd
import csv
from random import shuffle

In [4]:
def traduire(seq,strand):
    nucleotides = ''
    if strand:
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'a'
            elif lettre=='C':
                nucleotides=nucleotides+'c'
            elif lettre=='G':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    else:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'u'
            elif lettre=='C':
                nucleotides=nucleotides+'g'
            elif lettre=='G':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    return nucleotides

### Génération des bdd (ne pas exécuter)

In [4]:
parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")
dict_fasta = dict([(seq.id, seq) for seq in parser])

In [91]:
# base de données rise_human_targeted
bdd = pd.read_csv("rise_human_targeted.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

# Ne garder que les interactions miRNA - mRNA
arg_mi = np.argwhere(arrbdd[:,14]=='miRNA')[:,0]
arg_m = np.argwhere(arrbdd[:,15]=='protein_coding')[:,0]
args = [k for k in arg_mi if k in arg_m]
args = np.array(args)
arrbdd = arrbdd[args]

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'chrM') & (arrbdd[:,3] != 'chrM'))
arrbdd = arrbdd[arg_sansM[:,0]]
#enlever les champs vides
arg_sansM = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_sansM))
varrbdd = arrbdd[arg_sansM[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    varrbdd[i,1] = int(varrbdd[i,1])-1
    varrbdd[i,2] = int(varrbdd[i,2])
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = milieu-50-1
    varrbdd[i,5] = milieu+50
bdd = varrbdd

11473


In [237]:
writer = csv.writer(open("positifs.csv", 'w',newline=''),delimiter="\t")
    
for i in range(len(bdd)):
    pos = bdd[i,:4]
    ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
    pos[0] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,8])
    ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
    seq = dict_fasta[ide][begin:end].format("fasta").split('\n')
    pos[1] = traduire(seq[1]+seq[2],bdd[i,9])
    w = len(pos[0])
    pos[2] = np.array(RNA.pfl_fold_up(pos[0],1,w,w))[1:,1]
    pos[3] = np.array(RNA.pfl_fold_up(pos[1],1,101,101))[1:,1]
    writer.writerow(pos)        


In [238]:
# base de données rise_human_targeted
bdd = pd.read_csv("rise_human_targeted.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)[:16650]

# Ne garder que les interactions miRNA - mRNA
arg_mi = np.argwhere(arrbdd[:,14]=='miRNA')[:,0]
arg_m = np.argwhere(arrbdd[:,15]=='protein_coding')[:,0]
args = [k for k in arg_mi if k in arg_m]
args = np.array(args)
arrbdd = arrbdd[args]

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'chrM') & (arrbdd[:,3] != 'chrM'))
arrbdd = arrbdd[arg_sansM[:,0]]
#enlever les champs vides
arg_sansM = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_sansM))
varrbdd = arrbdd[arg_sansM[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    varrbdd[i,1] = int(varrbdd[i,1])-1
    varrbdd[i,2] = int(varrbdd[i,2])
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = milieu-50-1
    varrbdd[i,5] = milieu+50
bdd = varrbdd

10024


In [239]:
writer = csv.writer(open("negatifs.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)): 
    neg = bdd[i,:4]
    ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
    neg[0] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,8])
    ide, begin, end = bdd[i,3], bdd[i,4], bdd[i,5]
    begin = begin+80
    end = end+80
    sequ = dict_fasta[ide][begin:end].format("fasta").split('\n')
    neg[1] = traduire(sequ[1]+sequ[2],bdd[i,9])
    w = len(neg[0])
    neg[2] = np.array(RNA.pfl_fold_up(neg[0],1,w,w))[1:,1]
    neg[3] = np.array(RNA.pfl_fold_up(neg[1],1,101,101))[1:,1]
    writer.writerow(neg)

### Récupération et vérification des bases de données (exécuter à partir d'ici pour l'apprentissage)